In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import time
import re 
import datetime
import glob
import os

In [2]:
gu_list1 = ['종로구', '중구', '용산구', '성동구', '광진구',] 
gu_list2 = ['동대문구', '중랑구', '성북구', '강북구', '도봉구']
# gu_list3 = ['노원구', '은평구', '서대문구', '마포구', '양천구'] 
# gu_list4 = ['강서구', '구로구', '금천구', '영등포구', '동작구']
# gu_list5 = ['관악구', '서초구', '강남구', '송파구', '강동구']

gu_list = gu_list1 \
        + gu_list2
gu_list

['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구']

In [3]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

gift = '선물세트'

url_dict = {}
for region_gu in gu_list: 
    driver.get(f'https://www.daangn.com/search/{region_gu}%20{gift}')

    for i in range(20):
        try:
            driver.find_element_by_class_name("more-btn").click()
            time.sleep(1)
        except:
            break

    # get > url data 
    hrefs = driver.find_elements_by_class_name('articles-wrap > article')

    url_list = []
    for href in hrefs:
        
        url = href.find_element_by_tag_name('a').get_attribute('href')
        url_list.append(url)
        
        
    url_list = url_list[:-7]
    
    url_dict[region_gu] = list(set(url_list))
    
    print(f"{region_gu} 주소 개수:", len(url_list))
    print(f"{region_gu} 중복제거 주소 개수:", len(list(set(url_list))))
    
    # export table > url data 
#     current = datetime.datetime.now().replace(microsecond=0).strftime("%Y-%m-%d %H%M%S")
#     url_df = pd.Series(list(set(url_list)), name='url').to_frame()
#     url_df.to_excel(f"carrot_url_{region_gu}_{gift}_{current}.xlsx")
driver.quit()

# get > carrot data 


for region_gu in list(url_dict.keys()): 

    carrot_list = []

    # start time 
    start_time = datetime.datetime.now().replace(microsecond=0)
    start_time_str = start_time.strftime("%Y-%m-%d %H%M%S")
    print(f"{region_gu} 시작 :", start_time_str)

    for url in url_dict[region_gu]:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        
#         driver.get(url)
#         driver.implicitly_wait(10)
    #     time.sleep(1)

        try:
            num = int(url[-9:])
            region = soup.select('#region-name')[0].text

            title = soup.select('#article-title')[0].text

            categorys = soup.select('#article-category')[0].text.split("∙")
            category = categorys[0].strip()
            time1 = categorys[1].strip()

            try :
                price = soup.select('#article-price')[0].text.replace(",", "").strip()
                price = int(price.replace("원", ""))
            except:
                price = 0
            detail = soup.select('#article-detail > p')[0].text

            counts = soup.select("#article-counts")[0].text.split("∙")

            interest = int(counts[0].strip()[3:])
            chat = int(counts[1].strip()[3:])
            view = int(counts[2].strip()[3:])

            columns = [num, region, title, category, time1, price, detail, interest, chat, view]
            carrot_list.append(columns)

        except:
            pass

    

    # Dataframe
    columns = ['num', 'region', 'title', 'category', 'time', 'price', 'detail', 'interest', 'chat', 'view']
    df = pd.DataFrame(carrot_list, columns=columns) 
    print(f"{region_gu} 중복제거 전:", len(df))
    df = df.drop_duplicates()
    print(f"{region_gu} 중복제거 후:", len(df))

    # time
    finish_time = datetime.datetime.now().replace(microsecond=0)
    finish_time_str = finish_time.strftime("%Y-%m-%d %H%M%S")
    time_delta = finish_time - start_time
    print(f"{region_gu} 종료 :", finish_time_str)
    print(f"{region_gu} 소요시간 :", time_delta)
    print('-'*30)

    # export table > carrot data 
    file_name = f"carrot_list_{region_gu}_{gift}_{finish_time_str}.xlsx"
    df.to_excel(file_name, index=False)

    # export table > data list 
    data_list = pd.read_excel("data_list.xlsx")
    data_list = data_list.append({
        "file_name": file_name, 
        "start_time" : start_time,
        "finish_time" : finish_time, 
        "time_delta_seconds" : time_delta.total_seconds(), 
        "counts" : len(df)    
    }, ignore_index = True)

    data_list.to_excel("data_list.xlsx", index=False)
    
# driver.quit()

종로구 주소 개수: 241
종로구 중복제거 주소 개수: 240
중구 주소 개수: 246
중구 중복제거 주소 개수: 241
용산구 주소 개수: 242
용산구 중복제거 주소 개수: 237
성동구 주소 개수: 248
성동구 중복제거 주소 개수: 241
광진구 주소 개수: 245
광진구 중복제거 주소 개수: 239
동대문구 주소 개수: 248
동대문구 중복제거 주소 개수: 242
중랑구 주소 개수: 247
중랑구 중복제거 주소 개수: 243
성북구 주소 개수: 248
성북구 중복제거 주소 개수: 244
강북구 주소 개수: 245
강북구 중복제거 주소 개수: 231
도봉구 주소 개수: 245
도봉구 중복제거 주소 개수: 245
종로구 시작 : 2021-10-04 101852
종로구 중복제거 전: 238
종로구 중복제거 후: 238
종로구 종료 : 2021-10-04 101954
종로구 소요시간 : 0:01:02
------------------------------
중구 시작 : 2021-10-04 101956
중구 중복제거 전: 241
중구 중복제거 후: 241
중구 종료 : 2021-10-04 102057
중구 소요시간 : 0:01:01
------------------------------
용산구 시작 : 2021-10-04 102057
용산구 중복제거 전: 236
용산구 중복제거 후: 236
용산구 종료 : 2021-10-04 102157
용산구 소요시간 : 0:01:00
------------------------------
성동구 시작 : 2021-10-04 102157
성동구 중복제거 전: 238
성동구 중복제거 후: 238
성동구 종료 : 2021-10-04 102258
성동구 소요시간 : 0:01:01
------------------------------
광진구 시작 : 2021-10-04 102258
광진구 중복제거 전: 239
광진구 중복제거 후: 239
광진구 종료 : 2021-10-04 102359
광진구 소요시간 : 0:01:01
------

In [4]:
# get > carrot data 


for region_gu in list(url_dict.keys()): 

    carrot_list = []

    # start time 
    start_time = datetime.datetime.now().replace(microsecond=0)
    start_time_str = start_time.strftime("%Y-%m-%d %H%M%S")
    print(f"{region_gu} 시작 :", start_time_str)

    for url in url_dict[region_gu]:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")
        
#         driver.get(url)
#         driver.implicitly_wait(10)
    #     time.sleep(1)

        try:
            num = int(url[-9:])
            region = soup.select('#region-name')[0].text

            title = soup.select('#article-title')[0].text

            categorys = soup.select('#article-category')[0].text.split("∙")
            category = categorys[0].strip()
            time1 = categorys[1].strip()

            try :
                price = soup.select('#article-price')[0].text.replace(",", "").strip()
                price = int(price.replace("원", ""))
            except:
                price = 0
            detail = soup.select('#article-detail > p')[0].text

            counts = soup.select("#article-counts")[0].text.split("∙")

            interest = int(counts[0].strip()[3:])
            chat = int(counts[1].strip()[3:])
            view = int(counts[2].strip()[3:])

            columns = [num, region, title, category, time1, price, detail, interest, chat, view]
            carrot_list.append(columns)

        except:
            pass

    

    # Dataframe
    columns = ['num', 'region', 'title', 'category', 'time', 'price', 'detail', 'interest', 'chat', 'view']
    df = pd.DataFrame(carrot_list, columns=columns) 
    print(f"{region_gu} 중복제거 전:", len(df))
    df = df.drop_duplicates()
    print(f"{region_gu} 중복제거 후:", len(df))

    # time
    finish_time = datetime.datetime.now().replace(microsecond=0)
    finish_time_str = finish_time.strftime("%Y-%m-%d %H%M%S")
    time_delta = finish_time - start_time
    print(f"{region_gu} 종료 :", finish_time_str)
    print(f"{region_gu} 소요시간 :", time_delta)
    print('-'*30)

    # export table > carrot data 
    file_name = f"carrot_list_{region_gu}_{gift}_{finish_time_str}.xlsx"
    df.to_excel(file_name, index=False)

    # export table > data list 
    data_list = pd.read_excel("data_list.xlsx")
    data_list = data_list.append({
        "file_name": file_name, 
        "start_time" : start_time,
        "finish_time" : finish_time, 
        "time_delta_seconds" : time_delta.total_seconds(), 
        "counts" : len(df)    
    }, ignore_index = True)

    data_list.to_excel("data_list.xlsx", index=False)
    
# driver.quit()

종로구 시작 : 2021-10-03 101555
종로구 중복제거 전: 237
종로구 중복제거 후: 237
종로구 종료 : 2021-10-03 101729
종로구 소요시간 : 0:01:34
------------------------------
중구 시작 : 2021-10-03 101731
중구 중복제거 전: 238
중구 중복제거 후: 238
중구 종료 : 2021-10-03 101905
중구 소요시간 : 0:01:34
------------------------------
용산구 시작 : 2021-10-03 101906
용산구 중복제거 전: 238
용산구 중복제거 후: 238
용산구 종료 : 2021-10-03 102026
용산구 소요시간 : 0:01:20
------------------------------
성동구 시작 : 2021-10-03 102027
성동구 중복제거 전: 238
성동구 중복제거 후: 238
성동구 종료 : 2021-10-03 102158
성동구 소요시간 : 0:01:31
------------------------------
광진구 시작 : 2021-10-03 102159
광진구 중복제거 전: 240
광진구 중복제거 후: 240
광진구 종료 : 2021-10-03 102333
광진구 소요시간 : 0:01:34
------------------------------
동대문구 시작 : 2021-10-03 102333
동대문구 중복제거 전: 241
동대문구 중복제거 후: 241
동대문구 종료 : 2021-10-03 102509
동대문구 소요시간 : 0:01:36
------------------------------
중랑구 시작 : 2021-10-03 102509
중랑구 중복제거 전: 243
중랑구 중복제거 후: 243
중랑구 종료 : 2021-10-03 102648
중랑구 소요시간 : 0:01:39
------------------------------
성북구 시작 : 2021-10-03 102648
성북구 중복제거 전: 244
성북구 중